In [1]:
import json
from typing import Any, Dict, List, Tuple

In [2]:
import utils

In [3]:
utils.getNFTVolumes(28931584,32993741, 137)

getVolumes(): begin
getVolumes(): done


({'0x282d8efce846a88b159800bd4130ad77443fa1a1': {'0xa798fc8fa08c120e545a1c639ebe2506ba282961': 5.0,
   '0x30d1b625663e0da48379bf8ae53455a57d406011': 5.0,
   '0x4502f12c1b3c304ac3a7dae71e1b091d139a7e73': 5.0,
   '0x03894e05af1257714d1e06a01452d157e3a82202': 10.0,
   '0x546f3b8ad5dd18ed77820718cec56b26b8753a6c': 5.0,
   '0x70f1d5a5cf3e2a70a2169efd945cdd63e55d7b24': 2.0,
   '0x0e122f7d453e983ef0233af4e2af4cb053e5f075': 49.0,
   '0xfe7f1a646773c4056fc7b7a847ce4034228462d4': 10.0,
   '0x253ca313da088b70dcfeb20fe0533745a3fefc8a': 5.0,
   '0x7f0050fa5bdfe9388e705f56dfd79f113da1bd9a': 10.0,
   '0x2d8e318e7052b36e322781c68c835ed0a6c23a11': 5.0,
   '0x434a20a2888da5f9cc7e99f94fc022220c3e648a': 5.0,
   '0x3ff48c6bec9824939bd4e219c8ef52b8a49821a7': 5.0,
   '0x8d86a9098e2f898552cf8bd9608c6cb80688a3d7': 1.0,
   '0x8c57f836afc4a5fda8dd2fce53f3afe2440e3203': 5.0},
  '0x2791bca1f2de4661ed88a30c99a7a9449aa84174': {'0x6397829f2e9d997960fb4286531f6c8cf07a19c7': 5.0,
   '0x65f219fcf8c2de378f673981d2954c9ee